In [1]:
using Jutul, JutulDarcy, HYPRE, GLMakie, MAT, Statistics

import Jutul: find_enclosing_cell, plot_mesh_edges, replace_variables!
import JutulDarcy: table_to_relperm, add_relperm_parameters!, brooks_corey_relperm
import JutulDarcy: KilloughHysteresis, ReservoirRelativePermeabilities

In [2]:
vars = matread("E:/Ensemble-Model-Calibration-CO2/simulations3D_big/data_1272_128x128x16.mat")
perm_all = vars["perm"]
print("Perm all: ", size(perm_all))

Perm all: (1272, 128, 128, 16)

In [15]:
g = Jutul.gravity_constant
nx, ny, nz = 128, 128, 16
dx, dy, dz = 19.5312, 19.5312, 9.3750
Darcy, bar, psi, kg, meter, hour, day, year = si_units(:darcy, :bar, :psi, :kilogram, :meter, :hour, :day, :year)

cart_dims     = (nx, ny, nz)
physical_dims = (nx*dx, ny*dy, nz*dz)
mesh          = UnstructuredMesh(CartesianMesh(cart_dims, physical_dims))
nc            = number_of_cells(mesh)

# points = mesh.node_points
# for (i, pt) in enumerate(points)
#     y, x, z = pt
#     x_u = 0.5 * π * x / 1000.0
#     w = 0.2
#     dz = 0.05*x + 0.05*abs(x - 2500.0)+ w*(30*cos(2.0*x_u) + 20*sin(5.0*x_u))
#     points[i] = pt + [0, 0, dz]
# end

262144

In [25]:
# setup rock
poro = fill(0.2, nc)
perm = zeros(3, nc)
kx = vec(perm_all[114,:,:,:]) * Darcy ;
perm[1, :] = perm[2, :] = kx
perm[3, :] = 0.2 * kx

# setup fluid
so            = range(0, 1, 10)
sg            = range(0, 1, 50)
krog          = PhaseRelativePermeability(so, so.^2, label = :og)

tab_krg_drain = brooks_corey_relperm.(sg, n = 2, residual = 0.1)
tab_krg_imb   = brooks_corey_relperm.(sg, n = 3, residual = 0.25)

krg_drain     = PhaseRelativePermeability(sg, tab_krg_drain, label = :g)
krg_imb       = PhaseRelativePermeability(sg, tab_krg_imb, label = :g)

krg = (krg_drain, krg_imb)
H_g = KilloughHysteresis()
relperm = ReservoirRelativePermeabilities(g = krg, og = krog, hysteresis_g = H_g) ;

In [26]:
# setup reservoir
domain = reservoir_domain(mesh, permeability = perm, porosity = poro, temperature = convert_to_si(30.0, :Celsius))

# setup wells
Inj1 = setup_well(domain, [(32, 32, nz-3), (32, 32, nz-2), (32, 32, nz-1), (32, 32, nz)], name = :Injector1, simple_well=true)
Inj2 = setup_well(domain, [(32, 96, nz-3), (32, 96, nz-2), (32, 96, nz-1), (32, 96, nz)], name = :Injector2, simple_well=true)
Inj3 = setup_well(domain, [(96, 96, nz-3), (96, 96, nz-2), (96, 96, nz-1), (96, 96, nz)], name = :Injector3, simple_well=true)
Inj4 = setup_well(domain, [(96, 32, nz-3), (96, 32, nz-2), (96, 32, nz-1), (96, 32, nz)], name = :Injector4, simple_well=true)

# setup model and update with relperms
model  = setup_reservoir_model(domain, :co2brine, wells = [Inj1,Inj2,Inj3,Inj4], extra_out = false, co2_physics = :kvalue)
replace_variables!(model, RelativePermeabilities = relperm)
add_relperm_parameters!(model) ;

In [27]:
# initial state
p0 = zeros(nc)
depth = domain[:cell_centroids][3,:]
@. p0 = 250bar + depth * g * 1000.0

state0 = setup_reservoir_state(model, Pressure = p0, OverallMoleFractions = [1.0, 0.0])
parameters = setup_parameters(model) ;

In [28]:
plot_reservoir(model)

In [29]:
# boundary conditions
boundary = Int[]
for cell in 1:nc
    I, J, K = cell_ijk(mesh, cell)
    if I == 1 || I == nx
        push!(boundary, cell)
    end
end
bc = flow_boundary_condition(boundary, domain, p0[boundary], fractional_flow = [1.0, 0.0])
println("Boundary condition added to $(length(bc)) cells.")

Boundary condition added to 4096 cells.


In [43]:
# setup schedule
ramps = [1hour, 11hour, 12hour, (4day-24hour), (15day-4day)] #[1hour, 12hour, 24hour, 4day, 15day]
nTrup = length(ramps)

Tinj   = 5year
dTinj  = year/12
nTinj  = Int(Tinj / dTinj)

Tmon   = 500year
dTmon  = 20year
nTmon  = Int(Tmon / dTmon)

dt_inj = repeat([dTinj], nTinj)
dt_mon = repeat([dTmon], nTmon)
dt     = vcat(ramps, dt_inj, dt_mon)

inj_rate    = 0.5 * 1e9 / 686.5266 / year
rate_target = TotalRateTarget(inj_rate)
I_ctrl      = InjectorControl(rate_target, [0.0, 1.0], density = 686.5266)

controls = Dict()
controls[:Injector1] = I_ctrl
controls[:Injector2] = I_ctrl
controls[:Injector3] = I_ctrl
controls[:Injector4] = I_ctrl

forces_inj = setup_reservoir_forces(model, control = controls, bc = bc)
forces_mon = setup_reservoir_forces(model, bc = bc)
forces     = vcat(fill(forces_inj, nTrup), fill(forces_inj, nTinj), fill(forces_mon, nTmon))
println("$nTrup rampups + $nTinj report steps with injection, $nTmon report steps with migration.")

# run simulation
wd, states, t = simulate_reservoir(state0, model, dt, parameters = parameters, forces = forces,
                                   info_level = 1, 

                                   presolve_wells = true,
                                   linear_solver = :gmres, #:bicgstab
                                   precond = :ilu0, #:cpr
                                   max_nonlinear_iterations = 8,
                                   relaxation = true,
                                   tol_mb = 1e-4,
                                   rtol = 1e-3,
                                   max_timestep_cuts = 5)

5 rampups + 60 report steps with injection, 25 report steps with migration.
Jutul: Simulating 505 years, 2.149 weeks as 90 report steps
Step  1/90: Solving start to 1 hour, Δt = 1 hour 
Step  2/90: Solving 1 hour to 12 hours, Δt = 11 hours 
Step  3/90: Solving 12 hours to 1 day, 1 hour, Δt = 13 hours 
Step  4/90: Solving 1 day, 1 hour to 4 days, 1 hour, Δt = 3 days 
Step  5/90: Solving 4 days, 1 hour to 2 weeks, 1.042 day, Δt = 1 week, 4 days 
Convergence: Report step 5, mini-step #1 (23 hours, 10.98 minutes) failed to converge. Reducing mini-step.
Convergence: Report step 5, mini-step #5 (3 days, 12.5 hours) failed to converge. Reducing mini-step.
Convergence: Report step 5, mini-step #8 (4 days, 1.413 hour) failed to converge. Reducing mini-step.
Convergence: Report step 5, mini-step #9 (2 days, 42.4 minutes) failed to converge. Reducing mini-step.

In [134]:
plot_reservoir(model, states)

In [ ]:
inventory = co2_inventory(model, wd, states, t)
JutulDarcy.plot_co2_inventory(t, inventory)

***
# Functional